In [4]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain.globals import set_llm_cache
from langchain.cache import InMemoryCache, SQLiteCache
from langchain_core.output_parsers import StrOutputParser
from langchain_core.vectorstores import InMemoryVectorStore


from langchain_community.document_loaders import PyMuPDFLoader
from langchain_openai import OpenAIEmbeddings, ChatOpenAI

from ragas import evaluate
from ragas.metrics import faithfulness, answer_relevancy, context_precision, context_recall

from datasets import Dataset
from ragas.llms import LangchainLLMWrapper
import pandas as pd

import os
import re

from dotenv import load_dotenv
load_dotenv()

COLLECTION_NAME = "tax_law"
PERSIST_DIRECTORY = "tax"

In [3]:
def get_pdf_file_names(folder_path):
    """
    주어진 폴더 내에 있는 PDF 파일들의 이름을 리스트로 반환합니다.
    """
    import os
    
    try:
        all_files = os.listdir(folder_path)
        pdf_files = [file.replace(".pdf","") for file in all_files if file.lower().endswith('.pdf')]
        
        return pdf_files
    except FileNotFoundError:
        print(f"Error: 폴더 '{folder_path}'를 찾을 수 없습니다.")
        return []
    except Exception as e:
        print(f"Error: {e}")
        return []

In [4]:
def doc_split_by_pattern(full_text, split_pattern=None):
    """
    PDF 파일을 로드하고, 반복 텍스트 제거, 조항 기준으로 분할 및 연결 후 임베딩 준비.
    """
    import re
    
    if split_pattern: 
        chunks = re.split(split_pattern, full_text)

        connected_chunks = []
        current_chunk = ""
        is_buchik_section = False  # 부칙 여부 플래그

        for chunk in chunks:
            if re.search(r"\n\s*<부칙>", chunk):  # 부칙 시작 여부 확인
                is_buchik_section = True

            if chunk.startswith("제") and "조" in chunk:  # 새로운 조항 시작
                if is_buchik_section:  # 부칙 섹션이라면 접두사 추가
                    chunk = "부칙-" + chunk

                if current_chunk:  # 이전 조항 저장
                    connected_chunks.append(current_chunk.strip())
                current_chunk = chunk  # 새 조항으로 시작
            else:
                current_chunk += f" {chunk}"  # 기존 조항에 내용 추가

        if current_chunk:
            connected_chunks.append(current_chunk.strip())


        return connected_chunks, full_text
    else:
        return ([], '')


In [4]:
def get_vector_store(pdf_file_names):
    """
    PDF 파일들을 처리하여 임베딩을 Chroma Vector Store에 저장합니다.
    """
    import re
    from langchain_chroma import Chroma
    from langchain_core.documents import Document
    from langchain.document_loaders import PyMuPDFLoader

    all_docs = []
    
    for file_name in pdf_file_names:
        file_path = f"data/tax_law/{file_name}.pdf"  # PDF 파일 경로
        repeat_pattern = rf"법제처\s*\d+\s*국가법령정보센터\n{file_name.replace('_', ' ')}\n" 

        loader = PyMuPDFLoader(file_path)
        documents = loader.load()

        full_text = " ".join([doc.page_content for doc in documents])
        full_text = re.sub(repeat_pattern, "", full_text)
        
        split_pattern = r"\n(제\d+조[^\s]*)"
        
        chunks = re.split(split_pattern, full_text)

        connected_chunks = []
        current_chunk = ""
        is_buchik_section = False  # 부칙 여부 플래그

        for chunk in chunks:
            if re.search(r"\n\s*<부칙>", chunk):  # 부칙 시작 여부 확인
                is_buchik_section = True

            if chunk.startswith("제") and "조" in chunk:  # 새로운 조항 시작
                if is_buchik_section:  # 부칙 섹션이라면 접두사 추가
                    chunk = "부칙-" + chunk

                if current_chunk:  # 이전 조항 저장
                    connected_chunks.append(current_chunk.strip())
                current_chunk = chunk  # 새 조항으로 시작
            else:
                current_chunk += f" {chunk}"  # 기존 조항에 내용 추가

        if current_chunk:
            connected_chunks.append(current_chunk.strip())

        for idx, chunk in enumerate(connected_chunks):
            pattern = r"^제\d+조\d*(?:\([^)]*\))?"
            keyword = ""
            match = re.search(pattern, chunk)
            
            if match:
                keyword += match.group(0)
            
            keyword = re.sub(r'\s+', ' ', keyword)
            print(keyword)
            
            _doc = Document(metadata={"title": file_name, "keyword":keyword, "full_text": full_text[idx], }, page_content=chunk),
            all_docs.extend(_doc)
            
    # print(len(all_docs))
    # print(all_docs[1])
    # embedding_model = OpenAIEmbeddings(model="text-embedding-3-large")

    # vector_store = Chroma.from_documents(
    #     documents=all_docs,
    #     embedding=embedding_model,
    #     collection_name=COLLECTION_NAME,
    #     persist_directory=PERSIST_DIRECTORY
    # )
        
    # return vector_store
        
folder_path = "data/tax_law" 
pdf_files = get_pdf_file_names(folder_path)
get_vector_store(pdf_files)


제1조(과세대상과  세율)
제1조
제2조(비과세)
제3조(납세의무자)
제4조(과세시기)
제5조(제조로  보는 경우)
제6조(반출  등으로 보는 경우)
제7조(유흥음식요금을  전액 받은 것으로 보는 경우)
제8조(과세표준)
제9조(과세표준의  신고)
제10조(납부)
제10조
제10조
제21조
제10조
제10조
제11조(결정ㆍ경정결정  및 재경정)
제12조(수시부과)
제13조
제14조(미납세반출)
제15조(수출  및 군납 면세)
제16조(외교관  면세)
제17조(외국인전용판매장  면세)
제18조(조건부면세)
제19조(무조건  면세)
제19조
제20조(세액의  공제와 환급)
제20조(세액의  공제와 환급)
제20조
제20조
제21조(개업ㆍ폐업  등의 신고)
제22조(폐업으로  보지 아니하는 경우)
제23조(장부  기록의 의무)
제23조
제23조
제24조(권리ㆍ의무의  승계)
제26조(질문검사권)
제27조(영업정지  및 허가취소의 요구)
제29조(과태료)
제1조(시행일)
제2조(조건부면세에  관한 적용례)
제3조(조건부면세에  따른 세액의 환급 등에 관한 경과조치)

제1조(용도별  탄력세율을 적용하는 물품의 범위 등)
제2조(반출로  보지 아니하는 승인신청 등)
제2조
제3조(용기  대금 승인신청 등)
제4조(과세표준의  신고)
제5조(총괄납부의  승인신청 등)
제5조
제5조
제6조(미납세  및 면세 반출 승인신청 등)
제6조
제7조(반입신고  및 반입증명)
제7조
제8조(멸실  승인신청 등)
제9조(외교공관용  석유류 판매통보 등)
제10조(외국인전용판매장  지정신청)
제11조(면세물품  구입기록표 등)
제12조
제13조(유류  면세용도 사용보고 등)
제14조(면세물품  폐기 승인신청 등)
제14조
제15조(공제  및 환급 신청 등)
제16조(개업ㆍ폐업  등의 신고)
제18조
제19조

제1조(과세물품ㆍ과세장소  및 과세유흥장소의 세목등)
제2조(용어의  정의)
제2조
제3조(과세물품과  과세장소의 판정)
제4조(기준가격)
제4조
제5조
제6조(반출로  보지 않

In [48]:
# Prompt Template 생성
messages = [
        ("ai", """
        당신은 대한민국 세법에 대해 전문적으로 학습된 AI 도우미입니다. 저장된 세법 조항 데이터를 기반으로 사용자 질문에 답변하세요.

        - 모든 답변은 학습된 세법 데이터 내에서만 유효한 정보를 바탕으로 작성하세요. 데이터에 없는 내용은 추측하거나 임의로 생성하지 마세요.
        - 질문에 명확한 답변이 없거나 데이터 내에서 찾을 수 없는 경우, 정직하게 "잘 모르겠습니다."라고 말하고, 새로운 질문을 유도하세요.
        - 질문이 포함된 조항뿐 아니라, 필요 시 서로 연관된 다른 조항도 참고하여 답변의 정확성과 완성도를 높이세요.
        - 사용자가 이해하기 쉽게 답변을 구성하며, 중요한 키워드나 법 조항은 명확히 표시하세요.
        - 세법과 관련된 복잡한 질문에 대해서는 관련 조항 번호와 요약된 내용을 포함하여 답변을 제공하세요.
        
        추가 규칙:
        답변은 간결하고 명료하게 작성하되, 필요한 경우 관련 조항의 전문을 추가적으로 인용하세요.
        세법 용어를 사용자 친화적으로 설명하여 비전문가도 쉽게 이해할 수 있도록 하세요.
        질문을 완전히 이해하기 어렵거나 모호할 경우, 사용자가 구체적으로 질문을 다시 작성할 수 있도록 유도하는 후속 질문을 하세요.

	답변 후, 사용자에게 필요할 것 같은 정보를 바탕으로 두 가지 후속 질문을 제안하세요. 각 질문의 앞뒤에 한 줄씩 띄어쓰기를 하세요. 이 질문은 원래 주제와 관련된 내용이어야 합니다.
	특정 법률 조항이나 제도가 언급될 경우, 근거가 되는 세법 조문, 시행령, 또는 관련 자료를 명시합니다.
        모든 답변은 사용자에게 법적 조언이 아닌 정보 제공 목적으로 작성된 것임을 명확히 합니다. 
	{context}")"""
        ),
        ("human", "{question}"),
]
prompt_template = ChatPromptTemplate(messages)
# 모델
model = ChatOpenAI(model="gpt-4o")

# output parser
parser = StrOutputParser()

# Chain 구성 retriever(관련문서 조회) -> prompt_template(prompt 생성) -> model(정답) -> output parser
chain = {"context":retriever, "question": RunnablePassthrough()} | prompt_template | model | parser


In [10]:
# chain.invoke("개별소비세법이 뭐야?")
# chain.invoke("개별소비세법이 제1조가 뭐야")
# chain.invoke("교통_에너지_환경세법 뭐야?")
chain.invoke("조세범 처벌절차법의 정의해줘")


'조세범 처벌절차법에서 사용하는 용어의 정의는 다음과 같습니다:\n\n1. **조세범칙행위**: 이는 「조세범 처벌법」 제3조부터 제16조까지의 죄에 해당하는 위반행위를 말합니다.\n   \n2. **조세범칙사건**: 조세범칙행위의 혐의가 있는 사건을 의미합니다.\n\n3. **조세범칙조사**: 세무공무원이 조세범칙행위 등을 확정하기 위하여 조세범칙사건에 대하여 행하는 조사활동을 말합니다.\n\n4. **세무공무원**: 세무에 종사하는 공무원으로서, 지방국세청 소속 공무원이나 세무서 소속 공무원의 경우 각각 소속 지방국세청장의 제청으로 해당 지방국세청이나 세무서의 소재지를 관할하는 지방검찰청의 검사장이 지명하는 공무원을 의미합니다.\n\n이 정의들은 조세범 처벌절차법 제2조에 명시되어 있습니다. 추가적으로 궁금한 점이 있으면 말씀해 주세요!'